In [1]:
import numpy as np
import pandas as pd

import mlflow
import mlflow.sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv").drop(columns=["tweet_id"])

df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [5]:
x = df["sentiment"].isin(["sadness", "happiness"])
df = df[x]
df.sample(5)

,sentiment,content
29100,happiness,yay bannerbomb wii finally out run homebrew wii
35836,happiness,yinawu happy mother s day http bit ly ijm my f...
162,sadness,still missing husband really want home
14246,sadness,home ill work man flu hell
39773,happiness,agoodstein timknapton good role model like yin...


In [6]:
df.shape

(10374, 2)

In [7]:
df["sentiment"].replace({"sadness": 0, "happiness": 1}, inplace=True)

C:\Users\Acer\AppData\Local\Temp\ipykernel_8504\252988709.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["sentiment"].replace({"sadness": 0, "happiness": 1}, inplace=True)
C:\Users\Acer\AppData\Local\Temp\ipykernel_8504\252988709.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["sentim

In [8]:
df.sample(5)

,sentiment,content
38726,1,littlefletcher saw play here amazing
12835,1,yay im ride home take tha yellow limo super gay
32130,1,michaelni ooh love sweet potato fry definitely go
17601,0,jaredque get go home
32293,0,mortuary tweet making night bb girl


In [9]:
# BOW

In [10]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df["content"])
y = df["sentiment"]

In [11]:
test_size=0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

In [12]:
import dagshub
import mlflow

mlflow.set_tracking_uri('https://dagshub.com/pxxthik/MlOps-mini-project.mlflow')
dagshub.init(repo_owner='pxxthik', repo_name='MlOps-mini-project', mlflow=True)

mlflow.set_experiment("LR baseline")

Accessing as pxxthik

Initialized MLflow to track repo "pxxthik/MlOps-mini-project"

Repository pxxthik/MlOps-mini-project initialized!

<Experiment: artifact_location='mlflow-artifacts:/404c051ca43941838eb0afe93fe83bbd', creation_time=1749807762151, experiment_id='1', last_update_time=1749807762151, lifecycle_stage='active', name='LR baseline', tags={}>

In [13]:
with mlflow.start_run():

    # log preprocessing params
    mlflow.log_param("vectorizer", "BoW")
    mlflow.log_param("max_features", 1000)
    mlflow.log_param("test_size", test_size)

    # model building and training
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # log model parameter
    mlflow.log_param("model_name", "LogisticRegression")

    # model eval
    y_pred = model.predict(X_test)

    # log model metrics
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("precision", precision_score(y_test, y_pred))
    mlflow.log_metric("recall", recall_score(y_test, y_pred))
    mlflow.log_metric("f1", f1_score(y_test, y_pred))

    # log model
    mlflow.sklearn.log_model(model, "model")

    # save and log notebook
    import os
    notebook_path = "exp1_baseline_model.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)

    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Precision: {precision_score(y_test, y_pred)}")
    print(f"Recall: {recall_score(y_test, y_pred)}")
    print(f"F1: {f1_score(y_test, y_pred)}")

2025/06/13 16:27:17 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Accuracy: 0.7773493975903615
Precision: 0.7692307692307693
Recall: 0.7783251231527094
F1: 0.7737512242899118


2025/06/13 16:27:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run bustling-robin-977 at: https://dagshub.com/pxxthik/MlOps-mini-project.mlflow/#/experiments/1/runs/e7c74fb1cae94380a4df2a3996404ed8.
2025/06/13 16:27:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/pxxthik/MlOps-mini-project.mlflow/#/experiments/1.
